In [ ]:
import uproot
import pandas as pd
import matplotlib.pyplot as plt

# Open the ROOT file (update the filename as needed)
file = uproot.open("PhaseSpaceSimulation.root")

# Print the keys in the ROOT file to see what objects it contains:
print("Keys in the ROOT file:", file.keys())

# For example, if the file contains a TTree called "DecayTree", you can access it:
tree = file['PhaseSpaceTree;1']

# List available branches in the tree:
print("Branches in the tree:")
for var in tree.keys():
    print(var)

# Read selected branches into a pandas DataFrame:
# Replace these branch names with correct ones per the corresponding LHCb dataset documentation.
branches_to_load = list(tree.keys())  # adjust as necessary
df = tree.arrays(branches_to_load, library="pd")
print(df.head())

In [ ]:
data = df['H1_PX']

# Create a histogram of the data using 30 bins
plt.hist(data, bins=30, color='blue', edgecolor='black', alpha=0.7)

# Label the axes and set the title
plt.xlabel('H1_PX')
plt.ylabel('Frequency')

# Display the plot
plt.show()

In [ ]:
def hist(var):
    plt.hist(df[var], bins=30, color='blue', edgecolor='black', alpha=0.7)

    # Label the axes and set the title
    plt.xlabel(var)
    plt.ylabel('Frequency')
    
    # Display the plot
    plt.show()

In [ ]:
df['H1_P'] = (df['H1_PX']**2 + df['H1_PY']**2 + df['H1_PZ']**2)**(1/2)

plt.hist(df['H1_P'], bins=30, color='blue', edgecolor='black', alpha=0.7)

# Label the axes and set the title
plt.xlabel('H1 Magnitute of Momentum')
plt.ylabel('Frequency')

# Display the plot
plt.show()

In [ ]:
m_kaon = 493.677 #MeV
df['H1_E'] = (df['H1_P']**2 + m_kaon**2)**(1/2)

plt.hist(df['H1_E'], bins=30, color='blue', edgecolor='black', alpha=0.7)

# Label the axes and set the title
plt.xlabel('H1 Energy')
plt.ylabel('Frequency')

# Display the plot
plt.show()

In [ ]:
df['H2_P'] = (df['H2_PX']**2 + df['H2_PY']**2 + df['H2_PZ']**2)**(1/2)
df['H3_P'] = (df['H3_PX']**2 + df['H3_PY']**2 + df['H3_PZ']**2)**(1/2)
df['H1_E'] = (df['H2_P']**2 + m_kaon**2)**(1/2)
df['H1_E'] = (df['H3_P']**2 + m_kaon**2)**(1/2)

In [ ]:
df['B_E'] = df['H1_E'] + df['H2_E'] + df['H3_E']
df['B_PX'] = df['H1_PX'] + df['H2_PX'] + df['H3_PX']
df['B_PY'] = df['H1_PY'] + df['H2_PY'] + df['H3_PY']
df['B_PZ'] = df['H1_PZ'] + df['H2_PZ'] + df['H3_PZ']
df['B_P'] = (df['B_PX']**2 + df['B_PY']**2 + df['B_PZ']**2)**(1/2)

df['B_inv_mass'] = (df['B_E']**2 - df['B_P']**2)**(1/2)

plt.hist(df['B_inv_mass'], bins=30, color='blue', edgecolor='black', alpha=0.7)

# Label the axes and set the title
plt.xlabel('B Meson Mass')
plt.ylabel('Frequency')

# Display the plot
plt.show()